In [81]:
import json
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Мои переписки в телеграм 

In [82]:
def load_telegram_results(json_fname='data/1.json'):
    with open(json_fname) as f:
        r = json.load(f)
    chats = []
    for chat in r['chats']['list']:
        if 'name' not in chat:
            chat.pop('messages')
            print("Ignore chat", chat.keys())
            print(chat)
            continue
        #print(f'***  Process chat "{chat["name"]}" ***')
        prev_id = None
        dialog = []
        seq = []
        for m in chat['messages']:
            if 'text' not in m or 'from_id' not in m:
                continue
            
            text = m['text'] #.strip()
            if isinstance(text, list):
                t_out = []
                for s in text:
                    if isinstance(s, str):
                        t_out.append(s)
                text = ' '.join(t_out).strip()
            assert isinstance(text, str), m
            
            if not text or len(text) < 3:
                continue
            if m['from_id'] != prev_id and len(seq)>0:
                if prev_id is not None:
                    dialog.append('\n'.join(seq))
                prev_id = m['from_id']
                seq = []
            
            if not isinstance(text, str):
                continue 
            seq.append(text)
        if len(dialog) > 0:
            chats.append(dialog)
    return chats
telegram_data = load_telegram_results('data/result.json')

Ignore chat dict_keys(['type', 'id'])
{'type': 'saved_messages', 'id': 109109025}


In [83]:
total_len = 0
for d in telegram_data:
    total_len+=sum(len(seq.split())for seq in d)
total_len

1292093

# База комментариев с пикабу

In [84]:
picabu_df = pd.read_csv('data/russian_comments_from_2ch_pikabu.csv', delimiter=',')[['comment', 'toxic']]
picabu_df.dataframeName = 'russian_comments_from_2ch_pikabu.csv'
nRow, nCol = picabu_df.shape
print(f'There are {nRow} rows and {nCol} columns')
picabu_df.head()

There are 14412 rows and 2 columns


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


# База Диалогов с толоки

In [85]:
persona = pd.read_csv('data/TlkPersonaChatRus/dialogues.tsv', sep='\t')['dialogue']
persona[0]

'<span class=participant_2>Пользователь 2: Привет) расскажи о себе</span><br /><span class=participant_1>Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось<br />)</span><br /><span class=participant_2>Пользователь 2: Что читаешь? Мне нравится классика</span><br /><span class=participant_2>Пользователь 2: Я тоже люблю пообщаться</span><br /><span class=participant_1>Пользователь 1: Люблю животных, просто обожаю, как и свою работу)</span><br /><span class=participant_1>Пользователь 1: Я фантастику люблю</span><br /><span class=participant_2>Пользователь 2: А я выращиваю фиалки</span><br /><span class=participant_2>Пользователь 2: И веду здоровый и активный образ жизни!</span><br /><span class=participant_1>Пользователь 1: Ух ты, интересно.</span><br /><span class=participant_2>Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду<br />..</span><br /><span class=participant_1>Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим<br />дру

In [86]:
from html.parser import HTMLParser
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.dialog = []
        self.last_p=None
        self.seq = []
        
    def handle_data(self, data):
        #print("Encountered some data  :", data)
        if data.startswith('Пользователь'):
            p, n, part = data.split(' ', 2)
            assert p == 'Пользователь', data
        else:
            n = self.last_p
            part = data
        if self.last_p != n and self.last_p != None:
            self.dialog.append('\n'.join(self.seq))
            self.seq = []
        self.last_p = n
        self.seq.append(part)
        
parser = MyHTMLParser()
for p in persona:
    parser.feed(p)
parser.dialog[2]

'Что читаешь? Мне нравится классика\nЯ тоже люблю пообщаться'

In [87]:
TikPersonaChatRus = parser.dialog

# Обучение
Для начала будем использовать только диалоги. А именно базу из телеграмма и базу TikPersonaChatRus 
 

In [88]:
import sentencepiece
import torch
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruT5-base')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/sberbank-ai/ruT5-base/resolve/main/config.json from cache at /home/mitrofanov-aa/.cache/huggingface/transformers/15f5f9138b337892ef8dadfab622952847d6eb4b5985a825847fbaa18538bae9.d9b947fcfcea30df5eb71effa3afe0a2e3da3535463b97a14c3c0401ac680b99
Model config T5Config {
  "_name_or_path": "/home/jovyan/models/t5/t5_base_org",
  "_num_labels": 2,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specifi

In [89]:
tokenizer(['Пользователь: че каво', 'Курс: нормэ'])

{'input_ids': [[25089, 23, 4156, 24, 7, 608, 2], [11219, 23, 3318, 751, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [90]:
tokenizer('Пользователь:', add_special_tokens=False)

{'input_ids': [25089, 23], 'attention_mask': [1, 1]}

In [91]:
tokenizer('Курс:',  add_special_tokens=False)

{'input_ids': [11219, 23], 'attention_mask': [1, 1]}

In [92]:
def get_substrings_items(dialogs, user_prefix_len, assistant_prefix_len, max_len=600, verbose=False):
    d_se = []
    for d_id, d in enumerate(dialogs):
        if verbose:
            print(f"{d_id=}, {d=}")
        for s_id in range(len(d)):
            if verbose:
                print(f"{s_id=}, {d[s_id]=}")
            curr_len=0 # len(self.up_tokenized)
            person=True
            #d_se = []
            for s_e_id in range(s_id, len(d)):
                curr_len += user_prefix_len if person else assistant_prefix_len
                curr_len += len(d[s_e_id])
                if verbose:
                    print(f"{s_e_id=}, {person=}, {curr_len=}, {d[s_e_id]=}")
                if curr_len >= max_len:
                    if verbose:
                        print(f'{curr_len=} greate than max_len.')
                    break
                if not person:
                    d_se.append((d_id, s_id, s_e_id))
                person = not person
    return d_se
                     
get_substrings_items([[[1, 2, 3], [4,5], [6,7,8,9,0], [1,2], [3], [6,7,8], [8,9,0,0,1]]], 1, 1, max_len=7, verbose=True)

d_id=0, d=[[1, 2, 3], [4, 5], [6, 7, 8, 9, 0], [1, 2], [3], [6, 7, 8], [8, 9, 0, 0, 1]]
s_id=0, d[s_id]=[1, 2, 3]
s_e_id=0, person=True, curr_len=4, d[s_e_id]=[1, 2, 3]
s_e_id=1, person=False, curr_len=7, d[s_e_id]=[4, 5]
curr_len=7 greate than max_len.
s_id=1, d[s_id]=[4, 5]
s_e_id=1, person=True, curr_len=3, d[s_e_id]=[4, 5]
s_e_id=2, person=False, curr_len=9, d[s_e_id]=[6, 7, 8, 9, 0]
curr_len=9 greate than max_len.
s_id=2, d[s_id]=[6, 7, 8, 9, 0]
s_e_id=2, person=True, curr_len=6, d[s_e_id]=[6, 7, 8, 9, 0]
s_e_id=3, person=False, curr_len=9, d[s_e_id]=[1, 2]
curr_len=9 greate than max_len.
s_id=3, d[s_id]=[1, 2]
s_e_id=3, person=True, curr_len=3, d[s_e_id]=[1, 2]
s_e_id=4, person=False, curr_len=5, d[s_e_id]=[3]
s_e_id=5, person=True, curr_len=9, d[s_e_id]=[6, 7, 8]
curr_len=9 greate than max_len.
s_id=4, d[s_id]=[3]
s_e_id=4, person=True, curr_len=2, d[s_e_id]=[3]
s_e_id=5, person=False, curr_len=6, d[s_e_id]=[6, 7, 8]
s_e_id=6, person=True, curr_len=12, d[s_e_id]=[8, 9, 0, 0, 1]


[(0, 3, 4), (0, 4, 5)]

In [93]:
class DialogDataSet:
    def __init__(self, tokenizer, dialogs, user_prefix='Пользователь: ', assistant_prefix='Курс: ', max_len=40):
        self.dialogs = dialogs
        self.tokenizer = tokenizer
        self.user_prefix = user_prefix
        self.assistant_prefix = assistant_prefix
        self.eos_token_id = tokenizer.eos_token_id
        with tokenizer.as_target_tokenizer():
            self.up_tokenized = tokenizer(user_prefix, add_special_tokens=False)['input_ids']
            self.ap_tokenized = tokenizer(assistant_prefix, add_special_tokens=False)['input_ids']
            self.tokenized = [[s for s in tokenizer(d, add_special_tokens=False)['input_ids'] if s != [8, 0]] for d in dialogs ]
        
        self.d_s_l = get_substrings_items(self.tokenized, 
                                          len(self.up_tokenized), 
                                          len(self.ap_tokenized), 
                                          max_len=max_len)
        print(f"Dataset have {len(self.d_s_l)} examples")
    
    def get_full_seq_context(self, seq):
        return 
    
    def __getitem__(self, index):
        d, s, l = self.d_s_l[index]
        input_ids = self.up_tokenized + \
            [t for i, seq in enumerate(self.tokenized[d][s:l]) \
             for t in seq + [self.eos_token_id] + (self.ap_tokenized if i %2==0 else self.up_tokenized)]
        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(self.tokenized[d][l] + [self.eos_token_id])
        return {"input_ids":input_ids,
                "labels": labels, }
                #'attention_mask': torch.ones_like(input_ids)}
    
    def __len__(self):
        return len(self.d_s_l)
    
    def collate(self, items):
        pad_id = tokenizer.pad_token_id
        input_ids = [item['input_ids'] for item in items]
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=pad_id)
        labels = [item['labels'] for item in items]
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=pad_id)
        #attention_mask = [item['attention_mask'] for item in items]
        #torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0),
        return {"input_ids": input_ids,
                "labels": labels,
                "attention_mask": (input_ids != pad_id).float(), 
                'decoder_attention_mask': (labels != pad_id).float()
               }
        

In [94]:
class PairDataSet:
    def __init__(self, tokenizer, dialogs, max_len=40):
        self.dialogs = dialogs
        self.tokenizer = tokenizer
        self.max_tokens = max_len
        self.eos_token_id = tokenizer.eos_token_id
        
        with tokenizer.as_target_tokenizer():
            tokenized = [tokenizer(d, add_special_tokens=True, padding="max_length", truncation=True, max_length=max_len, ret) for d in dialogs]
        #self.input_ids = torch.cat((d['input_ids'] for d in tokenized)) 
        
        self.d_s_l = []
        print(f"Dataset have {len(self.d_s_l)} examples")
    
    def __getitem__(self, index):
        d, s, l = self.d_s_l[index]
        
        dialog = self.tokenized[d]
                                
        first = 
        second = self.tokenized[d]
        item = {'input_ids': dialog['input_ids'][s],
                'attention_mask': dialog['attention_mask'][s],
                'labels': dialog['input_ids'][l],
                'decoder_attention_mask': dialog['attention_mask'][l]}
        return item

    def __len__(self):
        return len(self.d_s_l)
    


SyntaxError: invalid syntax (874700275.py, line 20)

In [15]:
TikPersonaChatRus[:10]

['Привет) расскажи о себе',
 'Привет) под вкусный кофеек настроение поболтать появилось\n)',
 'Что читаешь? Мне нравится классика\nЯ тоже люблю пообщаться',
 'Люблю животных, просто обожаю, как и свою работу)\nЯ фантастику люблю',
 'А я выращиваю фиалки\nИ веду здоровый и активный образ жизни!',
 'Ух ты, интересно.',
 'Ты случайно не принц на белом коне? Я его очень жду\n..',
 'А у меня из хобби каждую неделю тусить с моим лучшим\nдругом)\nПривет!',
 'Привет,Как жизнь?',
 'Отлично) Солнышко светит, птички поют!']

In [16]:
# small_train_ds = DialogDataSet(tokenizer, [TikPersonaChatRus[:1000]])
# super_small = DialogDataSet(tokenizer, [TikPersonaChatRus[:100]])
# len(small_train_ds)

In [17]:
# my_telega_small = DialogDataSet(tokenizer, telegram_data[0:1])

In [18]:
persona_full = DialogDataSet(tokenizer, [TikPersonaChatRus])

In [19]:
# my_telega_full = DialogDataSet(tokenizer, telegram_data)

In [20]:
# train_full = DialogDataSet(tokenizer, [TikPersonaChatRus, *telegram_data])

In [72]:
tokenizer(telegram_data[0])

{'input_ids': [[29, 2861, 275, 4103, 73, 7283, 18205, 16690, 6, 8146, 2631, 285, 189, 1133, 223, 10, 858, 416, 486, 7553, 7, 8986, 27, 13481, 9, 624, 3, 1084, 17303, 3, 6997, 8, 23168, 4, 21900, 4293, 5823, 1961, 1401, 22, 6015, 58, 4737, 19, 9, 549, 1860, 241, 3167, 2], [2331, 113, 259, 43, 10, 13011, 40, 313, 2020, 277, 6, 2861, 10581, 82, 1674, 10013, 358, 2], [4378, 370, 171, 14, 50, 920, 59, 27, 5, 31, 579, 3530, 502, 10441, 2011, 877, 10374, 2], [118, 16215, 106, 29462, 44, 4827, 6, 27495, 13, 25, 2], [16, 22, 29462, 25, 512, 90, 272, 69, 16215, 123, 25, 2], [11, 6363, 5, 29600, 31, 8, 13474, 173, 481, 19872, 68, 30187, 798, 17, 1672, 7, 7, 1102, 46, 1018, 2], [1893, 5797, 35, 10, 17446, 27, 17, 1355, 591, 2707, 34, 6, 5, 168, 3905, 3, 503, 68, 16430, 1166, 2544, 258, 2], [165, 86, 40, 2], [3899, 10, 24650, 1338, 1205, 68, 1261, 4292, 6, 4279, 5, 168, 1018, 15014, 874, 2], [48, 16, 35, 303, 4416, 7, 25, 70, 4771, 115, 9, 6, 24557, 588, 7989, 25, 2], [8, 18408, 68, 1261, 1364, 6, 

In [69]:
my_telega_small = PairDataSet(tokenizer, telegram_data[0:1])
len(my_telega_small)

Dataset have 1 examples


1

In [41]:
for i in range(len(my_telega_small)):
    item = my_telega_small[i]
    w = tokenizer.decode(item['labels'], skip_special_tokens=True)
    if len(w) == 0:
        print(tokenizer.decode(item['labels']))
        print(i, item)

In [42]:
print(tokenizer.convert_ids_to_tokens(my_telega_small[-1]['input_ids']))
print(tokenizer.convert_ids_to_tokens(my_telega_small[-1]['labels']))

['▁О', ',', '▁вроде', '▁слышал', '</s>']
['▁Там', '▁молодой', '▁де', 'пар', 'дь', 'е', '▁А', '▁на', '▁английском', '▁он', '▁b', 'im', 'bo', 'land', '▁Б', 'л', '▁Свет', '▁только', '▁что', '▁вы', 'ко', 'ю', 'чили', '▁Через', '▁5', '▁секунд', '▁включил', 'и', '▁Что', '▁за', '▁при', 'кол', 'ы', '</s>']


In [43]:
print(tokenizer.convert_ids_to_tokens(my_telega_small[0]['input_ids']))
print(tokenizer.convert_ids_to_tokens(my_telega_small[0]['labels']))

['▁из', '▁ча', 'та', '▁ха', 'ка', 'тона', '▁=)', '▁Решил', '▁в', 'бросить', '▁тему', '▁т', 'к', '▁скорее', '▁всего', '▁не', '▁буду', '▁её', '▁делать', '▁Дев', 'а', 'йс', '▁как', '▁маска', '▁на', '▁лицо', ',', '▁внутри', '▁микрофон', ',', '▁наружу', '▁', 'динамик', '.', '▁Автомат', 'ически', '▁фильтр', 'ует', '▁речь', '▁за']
['▁Ах', 'ах', '▁Ну', '▁это', '▁не', '▁бесполезно', '!', '▁От', 'зов', 'ись', '▁в', '▁ча', 'тике', '▁про', '▁кино', '▁пл', 'з', '</s>']


In [44]:
print(tokenizer.convert_ids_to_tokens(my_telega_small[101]['input_ids']))
print(tokenizer.convert_ids_to_tokens(my_telega_small[101]['labels']))

['▁Спасибо', '▁Если', '▁хочешь', '▁я', '▁могу', '</s>']
['▁я', '▁не', '▁хочу', '▁бух', 'ать', '▁я', '▁не', '▁хочу', '▁на', '▁море', '</s>']


In [48]:
model = AutoModelForSeq2SeqLM.from_pretrained('sberbank-ai/ruT5-base')
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"t5-sber-finetuned-small",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4, 
    do_eval=False,
    logging_steps = 500,
logging_strategy='steps')

loading configuration file https://huggingface.co/sberbank-ai/ruT5-base/resolve/main/config.json from cache at /home/mitrofanov-aa/.cache/huggingface/transformers/15f5f9138b337892ef8dadfab622952847d6eb4b5985a825847fbaa18538bae9.d9b947fcfcea30df5eb71effa3afe0a2e3da3535463b97a14c3c0401ac680b99
Model config T5Config {
  "_name_or_path": "/home/jovyan/models/t5/t5_base_org",
  "_num_labels": 2,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0

In [49]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=my_telega_small,
    data_collator=my_telega_small.collate,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
)

In [50]:
trainer.train()

***** Running training *****
  Num examples = 19106
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4780


Step,Training Loss
500,2.501100
1000,1.901100
1500,1.843300
2000,1.815500
2500,1.793300
3000,1.756500
3500,1.728900
4000,1.733300
4500,1.711500


Saving model checkpoint to t5-sber-finetuned-small/checkpoint-500
Configuration saved in t5-sber-finetuned-small/checkpoint-500/config.json
Model weights saved in t5-sber-finetuned-small/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-small/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-small/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-small/checkpoint-500/spiece.model
Saving model checkpoint to t5-sber-finetuned-small/checkpoint-1000
Configuration saved in t5-sber-finetuned-small/checkpoint-1000/config.json
Model weights saved in t5-sber-finetuned-small/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-sber-finetuned-small/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-sber-finetuned-small/checkpoint-1000/special_tokens_map.json
Copy vocab file to t5-sber-finetuned-small/checkpoint-1000/spiece.model
Saving model checkpoint to t5-sber-finetuned

TrainOutput(global_step=4780, training_loss=1.8552679596585708, metrics={'train_runtime': 1019.06, 'train_samples_per_second': 74.995, 'train_steps_per_second': 4.691, 'total_flos': 3074381000294400.0, 'train_loss': 1.8552679596585708, 'epoch': 4.0})

In [53]:
def dict_to_device(d, device):
    return {k: v.to(device) if isinstance(v, torch.Tensor) else v for k,v in d.items()}

with torch.no_grad():
    batch = my_telega_small.collate([my_telega_small[3]])
    out_ids = model.generate(input_ids=batch['input_ids'].cuda(), 
                               attention_mask=batch['attention_mask'].cuda(),
                            num_beams=4, do_sample=True)
        
    #print(out_ids)
    print(tokenizer.decode(batch['input_ids'].squeeze()).replace('</s>', '\n'))
    print("Курс: >>> ", tokenizer.decode(out_ids.squeeze(), skip_special_tokens=True))
    print('Лейбл >>> ', tokenizer.decode(batch['labels'].squeeze()).replace('</s>', '\n'))
    

ты кидал бумажку для участия в лотерее?

Курс: >>>  го в плашетку?
Лейбл >>>  что за бумажку? куда ее надо было кидать?



In [97]:
def interractive(model=model, tokenizer=tokenizer, user_prefix='Пользователь: ', assistant_prefix='Курс: '):
    context=''
    input_line = input('<<< ')
    with torch.no_grad():
        for i in range(10):
            if not input_line:
                break
            context+=user_prefix + input_line + ' </s> ' + assistant_prefix.strip()
            #print(context)
            t = tokenizer(context, add_special_tokens=False)
            #print(t)
            #print(tokenizer.convert_ids_to_tokens(t['input_ids']))
            input_ids = t['input_ids'][-40:]
            input_ids=torch.LongTensor(t['input_ids']).cuda().view(1,-1)
            attention_mask = torch.ones_like(input_ids).cuda()
            out_ids = model.generate(input_ids=input_ids, 
                           attention_mask=attention_mask,
                           num_beams=4, do_sample=True)
            s = tokenizer.decode(out_ids.cpu().squeeze(), skip_special_tokens=True)
            print('>>> ', s)
            context+=s.strip() + ' </s> '
            
            input_line = input('<<< ')

interractive()

<<<  Как тебя зовут?


>>>  Я буду дома)


<<<  Как тебя зовут?


>>>  Я не знаю, а ты где?


<<<  Твое имя какое?


...

<<<  Имя


>>>  Не знаю Где-то на улице


<<<  жопа с ручкой


>>>  го


KeyboardInterrupt: Interrupted by user